In [1]:
import torch
from torch.autograd import Variable
from visualization import make_dot

class OracleNet(torch.nn.Module):
    def __init__(self, D_in, GRU_out, D_out):
        """
        In the constructor we construct three nn.Linear instances that we will use
        in the forward pass.
        """
        super(OracleNet, self).__init__()
        #self.recurrent = torch.nn.GRU() ????
        self.input_linear = torch.nn.Linear(D_in+GRU_out, D_out)

    def forward(self, x):
        #x_vocab = some subset of x
        #out_gru = recurrent(x_vocab) or something
        #x_noVocab = some subset of x
        #x_actual = torch.cat((out_gru,x_noVocab),1)
        y_pred = self.input_linear(x).clamp(min=0)
        return y_pred

In [2]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, GRU_out, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs, and wrap them in Variables
x = torch.randn(N, D_in)
x2 = torch.randn(N,GRU_out)
x3 = Variable(torch.cat((x,x2),1))
y = Variable(torch.randn(N, D_out), requires_grad=False)

# Construct our model by instantiating the class defined above
model = OracleNet(D_in, GRU_out, D_out)

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x3)

    # Compute and print loss
    loss = criterion(y_pred, y)
    print(t, loss.data[0])

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 788.9090576171875
1 636.5570068359375
2 543.8738403320312
3 520.1136474609375
4 529.4099731445312
5 548.0252685546875
6 552.368408203125
7 539.6798095703125
8 516.9963989257812
9 501.5533142089844
10 499.0120849609375
11 505.4289245605469
12 511.6039733886719
13 510.7738342285156
14 502.95556640625
15 494.0292053222656
16 489.2355651855469
17 489.6820373535156
18 491.7207946777344
19 492.409912109375
20 490.4081726074219
21 487.2669372558594
22 483.7555847167969
23 481.30877685546875
24 479.9659423828125
25 479.72210693359375
26 479.87969970703125
27 479.6661682128906
28 478.7794189453125
29 477.5014953613281
30 476.3858337402344
31 475.7471923828125
32 475.61236572265625
33 475.6435241699219
34 475.5020751953125
35 475.0751037597656
36 474.5003967285156
37 474.00823974609375
38 473.7338562011719
39 473.64501953125
40 473.6076354980469
41 473.5048828125
42 473.3129577636719
43 473.0775451660156
44 472.87841796875
45 472.74810791015625
46 472.67437744140625
47 472.623046875
48 472.566

In [3]:
make_dot(y)

ExecutableNotFound: failed to execute ['dot', '-Tsvg'], make sure the Graphviz executables are on your systems' PATH